---

## 2024-1학기 인터랙티브 그래프마이닝

# ✅ COVID-19 가짜 뉴스 탐지 (Guide Code)

---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
cd '/content/drive/MyDrive/2024-1 graph-mining/kaggle'

/content/drive/MyDrive/2024-1 graph-mining/kaggle


In [ ]:
import pandas as pd
import numpy as np

# 1. Data Description
- id : claim id
- label : 0(normal) / 1(fake)
- published_date : 보도 날짜
- keybert_keywords : kebert를 통해 추출한 기사 키워드
- ner_keywords : ner를 통해 추출한 기사 키워드
- youtube0 ~ youtube9 : 해당 키워드로 유트브에 검색했을 때 나오는 상위 10개의 유투브의 제목, 설명 텍스트

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [ ]:
# train data: 5277개의 기사로 구성
train

,id,label,claim,published_date,keybert_keywords,ner_keywords,youtube0,youtube1,youtube2,youtube3,youtube4,youtube5,youtube6,youtube7,youtube8,youtube9
0,0,0,Did a Vermont Woman Post a Joke About Trump Br...,2018-01-25,"['trump', 'vermont', 'woman']","['vermont', 'trump', 'woman']",Citizen Trump A talk given at the University ...,News Donald Trump has a great woman problem N...,Trump Supporters Remain Despite Vulgar Slur J...,BREAKING Donald trump has a great woman proble...,Revolutionary War SNL The Patriots from New E...,World News Donald Trump has a great woman prob...,Latest News Donald Trump has a great woman pro...,AM Executive Session Gender Matters The U S Pr...,Trump Discusses Tax Bill Gives ;Total Support ...,Chicago s Women s March January This is what ...
1,1,1,Were Baseball Players Photographed Kneeling to...,2017-11-08,"['lynchings', 'baseball', 's']","['lynchings', 'baseball', 's']",Tony Shalhoub Still ;Monk ; after all these ye...,Lesson Class differences and racial variation ...,Mass Media Mass Culture and the Golden Age of ...,Mark W Bennett quot;Implicit Bias and the Law ...,Chris Hedges Writing as Resistance presented b...,Race in America FOCIS th Anniversary lecture s...,,,,
2,2,0,Wisconsin state Rep. John Nygren hits pay dirt...,2019-02-07,"['tax', 'wisconsin', 'dirt']","['rep', 'wisconsin', 'tax', 'wisconsin']",Pocan Shutdown Waste of Time Energy Pocan Shu...,Lawmaker This couldn ;t be more embarrassing f...,State Rep Bob Behning Creating teacher career ...,US GOV th,Representative John Macco of the th talks the ...,Democratic Senator Calls On Va Gov To Resign M...,Rep Sean Duffy introduces the Reciprocal Trade...,Rep Dan Caulkins on IL Minimum Wage Hike Stat...,Rep Weber Worried More Jobs Will Leave IL Rea...,Foxconn May Not Build B Wisconsin Plant Presid...
3,3,0,Progressive group accuses Senate splinter grou...,2017-11-04,"['donors', 'cashing', 'senate']","['cashing', 'senate', 'donors']",Roy Moore s Attorney Refers To News Anchor ;s ...,Documents that Changed the Way We Live Docume...,Time to Wake Up EPA Nominees Show Blatant Disr...,Училище по персонализирана медицина Панел Учи...,Conference Theme Panel Economics for a New Pro...,,,,,
4,4,0,Seal with Unusual Stripe Pattern Markings Spot...,2016-10-21,"['seal', 'washington', 'markings']","['seal', 'washington', 'markings']",Parking Lot Striping Garland Texas Asphalt st...,Learn about the MAHLE Original Gasket Line Bi...,Austroads Guide to Road Design Part Session of,I Major Deck and Superstructure Rehabilitation...,The Premier Parking Lot Striping Company in Au...,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5272,5272,0,Does Hillary Clinton want a $15 or $12 minimum...,2016-04-14,"['hillary', 'clinton']","['hillary clinton', 'hillary', 'clinton']",Hillary Clinton Addresses Her Losing Streak Co...,Stronger Together Hillary Clinton Donald Trum...,Love and Kindness Hillary Clinton Hillary Cli...,Hillary Clinton Interview at The Breakfast Clu...,Hillary Clinton Reveals Bill ;s Most Annoying ...,Stephen Interviews Hillary Clinton At Carnegie...,Hillary Clinton ;s entire Pennsylvania primary...,Hillary Clinton on NY Primary Win FULL SPEECH ...,Thank you New York Hillary Clinton In her New...,Hillary Clinton ;s full New York primary victo...
5273,5273,1,George Soros s son isn t married to Adam Schif...,2019-10-10,"['sister', 'married', 'son']","['sister', 'married', 'son']",Norman Lusts After His Mother Bates Motel Scen...,Cenk ve Cemre Evlendi Şeniz Çıldırdı Zalim İs...,Seven Year Old Mykal Michelle Harris on Mixed ...,My Sister Song CoComelon Nursery Rhymes amp; K...,quot;He annoys me so much quot; Naomi Osaka On...,MA SŒUR SE MARIE JUST MARRIED Lévanah amp;Fami...,Sheikh Mohammed bin Rashid ;s daugher Maryam m...,Amelia and Link Tell Meredith About the Baby G...,Tamron Hall Talks with Her Nephew About the Mu...,Thomas Doherty Juliet Doherty amp; Harry Jarvi...
5274,5274,0,Sanders: It's not impossible to get enough del...,2016-

In [ ]:
# test data: 1856개의 기사로 구성 (test에는 라벨값이 없으므로, 이를 예측하여 submission 해야 함.)
test

,id,claim,published_date,keybert_keywords,ner_keywords,youtube0,youtube1,youtube2,youtube3,youtube4,youtube5,youtube6,youtube7,youtube8,youtube9
0,0,According to the CDC so far this year Florida ...,2020-05-27,['pneumonia'],"['florida', 'pneumonia', 'cdc']",Watch Full Coronavirus Coverage May NBC News N...,Watch Full Coronavirus Coverage May NBC News N...,Ask the expert Here ;s why screening is a good...,MORE Questions for an Infectious Disease Speci...,Coronavirus and Cuba,Understanding and Managing Coronavirus in SNF ...,SARS CoV Pandemic Caltech Seminar Day Online S...,CAH Swingbed Initiative with Kerry Dunning Ke...,Diabetics and Novel Coronavirus ▻ Theresa De ...,CDPHE explains how it classifies COVID deaths ...
1,1,Claim that Rep. Alexandria Ocasio-Cortez tweet...,2020-06-23,"['november', 'elections', 'tweeted']","['covid', 'us', 'rep']",Maps show explosive growth of coronavirus in U...,Rep Raja Krishnamoorthi on Continued Fallout o...,Biologist says young people are fueling Covid ...,Former Allergan CEO on the most likely timelin...,Rep DeGette questions top U S health officials...,Rep Kevin McCarthy on Covid spikes reopening t...,Rep Kevin Brady on back to work bonus proposal...,Florida and Texas struggle with surging corona...,What flying in the U S amid the coronavirus pa...,U S Virus Surge Ample Opportunity for Covid to...
2,2,Models projecting COVID-19 deaths are talking ...,2020-05-05,"['deaths', 'mitigation']","['deaths', 'covid', 'mitigation']",President Donald Trump downplays models projec...,Trump keeps predicting coronavirus death tolls...,Trump Claims New Estimate Predicting More Coro...,Is COVID Seasonal View CME information and cl...,The Bubonic Plague and COVID – Two Diseases in...,Coronavirus outbreak B C reports new cases add...,Coronavirus Number of UK deaths decline from p...,Coronavirus model projects deaths in U S to ne...,Modeling COVID Transmission and Containment in...,UK records most Covid deaths in Europe but Raa...
3,3,A video has been viewed tens of thousands of t...,2020-05-02,"['instagram', 'facebook', 'tea']","['covid', 'uk', 'boris johnson']",Coronavirus Boris Johnson updates nation on Co...,Coronavirus How Boris Johnson government ;s po...,Boris Johnson unveils new COVID lockdown rules...,Coronavirus Public challenge Boris Johnson ;s ...,Coronavirus UK now ;past the peak ; says Boris...,UK PM Boris Johnson returns to work after reco...,Boris Johnson We are past the peak of disease ...,In full Boris Johnson says he won ;t lift UK c...,Boris Johnson s new stay alert message splits ...,Watch Back Boris Johnson Addresses UK On State...
4,4,A video shows how Indian police frightened tho...,2020-04-24,"['ambulance', 'police', 'frightened']","['covid', 'indian', 'ambulance']",Watch TN Police put lockdown violators in ambu...,Fake Covid patient TN Police releases awarenes...,TN Police ;Prank ; Lockdown Violators With Fak...,Police drone catches teens violating COVID loc...,India s mobile coronavirus testing van hits Ne...,Ambulance Traffic Jams At Moscow Hospitals As ...,;India ;s Covid recovery rate ; Union govt on ...,Police drone interrupts couple ;s romantic get...,Watch Tamil Nadu Police put lockdown violators...,Pak Reaction on Indian Police Prank Offenders ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851,1851,COMMENTARY: COVID-19 Diary Day 6: Clinicians' ...,2020-04-10,"['clinicians', 'diary']","['clinicians', 'diary']",Adding Services and Products,Adding Clinicians,Acting Director ;s Message April,Adding and Editing Account Users,Advice for New Therapists Subscribe now for m...,COVID Clinical Webinar,Diary of a Researcher Day April Libby Researc...,THE HARDEST SURGERY OPERATION I ;VE EVER EXPER...,Configuring Business Basics,Quick Navigational Tips
1852,1852,Facebook and Instagram posts shared thousands ...,2020-05-25,"['instagram', 'facebook', 'vaccines']","['instagram', 'facebook', 'vaccines']",Ousted vaccine chief testifies Americans deser...,Anti Vaxxers Could

In [ ]:
# test 데이터 내 1856개의 노드에 대해 예측된 라벨 (sample에서는 모두 0으로 초기화되어 있음.)
submission

,ID,Label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
1851,1851,0
1852,1852,0
1853,1853,0
1854,1854,0


# 2. Graph Example

In [2]:
!pip install torch==2.2.1+cu121 torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.3/757.3 MB 918.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━

In [3]:
import torch
print("PyTorch has version {}".format(torch.__version__))
# install dgl
# https://www.dgl.ai/pages/start.html

PyTorch has version 2.2.1+cu121


In [4]:
# Install DGL library
!pip install  dgl -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 64.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
from scipy.sparse import coo_matrix
import torch
import dgl

In [ ]:
# adjacency matrix 구축 예시: 같은 월에 작성된 claim끼리 연결
train['published_date'] = pd.to_datetime(train['published_date'], format='%Y-%m-%d', errors='raise')
train

,id,label,claim,published_date,keybert_keywords,ner_keywords,youtube0,youtube1,youtube2,youtube3,youtube4,youtube5,youtube6,youtube7,youtube8,youtube9
0,0,0,Did a Vermont Woman Post a Joke About Trump Br...,2018-01-25,"['trump', 'vermont', 'woman']","['vermont', 'trump', 'woman']",Citizen Trump A talk given at the University ...,News Donald Trump has a great woman problem N...,Trump Supporters Remain Despite Vulgar Slur J...,BREAKING Donald trump has a great woman proble...,Revolutionary War SNL The Patriots from New E...,World News Donald Trump has a great woman prob...,Latest News Donald Trump has a great woman pro...,AM Executive Session Gender Matters The U S Pr...,Trump Discusses Tax Bill Gives ;Total Support ...,Chicago s Women s March January This is what ...
1,1,1,Were Baseball Players Photographed Kneeling to...,2017-11-08,"['lynchings', 'baseball', 's']","['lynchings', 'baseball', 's']",Tony Shalhoub Still ;Monk ; after all these ye...,Lesson Class differences and racial variation ...,Mass Media Mass Culture and the Golden Age of ...,Mark W Bennett quot;Implicit Bias and the Law ...,Chris Hedges Writing as Resistance presented b...,Race in America FOCIS th Anniversary lecture s...,,,,
2,2,0,Wisconsin state Rep. John Nygren hits pay dirt...,2019-02-07,"['tax', 'wisconsin', 'dirt']","['rep', 'wisconsin', 'tax', 'wisconsin']",Pocan Shutdown Waste of Time Energy Pocan Shu...,Lawmaker This couldn ;t be more embarrassing f...,State Rep Bob Behning Creating teacher career ...,US GOV th,Representative John Macco of the th talks the ...,Democratic Senator Calls On Va Gov To Resign M...,Rep Sean Duffy introduces the Reciprocal Trade...,Rep Dan Caulkins on IL Minimum Wage Hike Stat...,Rep Weber Worried More Jobs Will Leave IL Rea...,Foxconn May Not Build B Wisconsin Plant Presid...
3,3,0,Progressive group accuses Senate splinter grou...,2017-11-04,"['donors', 'cashing', 'senate']","['cashing', 'senate', 'donors']",Roy Moore s Attorney Refers To News Anchor ;s ...,Documents that Changed the Way We Live Docume...,Time to Wake Up EPA Nominees Show Blatant Disr...,Училище по персонализирана медицина Панел Учи...,Conference Theme Panel Economics for a New Pro...,,,,,
4,4,0,Seal with Unusual Stripe Pattern Markings Spot...,2016-10-21,"['seal', 'washington', 'markings']","['seal', 'washington', 'markings']",Parking Lot Striping Garland Texas Asphalt st...,Learn about the MAHLE Original Gasket Line Bi...,Austroads Guide to Road Design Part Session of,I Major Deck and Superstructure Rehabilitation...,The Premier Parking Lot Striping Company in Au...,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5272,5272,0,Does Hillary Clinton want a $15 or $12 minimum...,2016-04-14,"['hillary', 'clinton']","['hillary clinton', 'hillary', 'clinton']",Hillary Clinton Addresses Her Losing Streak Co...,Stronger Together Hillary Clinton Donald Trum...,Love and Kindness Hillary Clinton Hillary Cli...,Hillary Clinton Interview at The Breakfast Clu...,Hillary Clinton Reveals Bill ;s Most Annoying ...,Stephen Interviews Hillary Clinton At Carnegie...,Hillary Clinton ;s entire Pennsylvania primary...,Hillary Clinton on NY Primary Win FULL SPEECH ...,Thank you New York Hillary Clinton In her New...,Hillary Clinton ;s full New York primary victo...
5273,5273,1,George Soros s son isn t married to Adam Schif...,2019-10-10,"['sister', 'married', 'son']","['sister', 'married', 'son']",Norman Lusts After His Mother Bates Motel Scen...,Cenk ve Cemre Evlendi Şeniz Çıldırdı Zalim İs...,Seven Year Old Mykal Michelle Harris on Mixed ...,My Sister Song CoComelon Nursery Rhymes amp; K...,quot;He annoys me so much quot; Naomi Osaka On...,MA SŒUR SE MARIE JUST MARRIED Lévanah amp;Fami...,Sheikh Mohammed bin Rashid ;s daugher Maryam m...,Amelia and Link Tell Meredith About the Baby G...,Tamron Hall Talks with Her Nephew About the Mu...,Thomas Doherty Juliet Doherty amp; Harry Jarvi...
5274,5274,0,Sanders: It's not impossible to get enough del...,2016-

In [ ]:
# adjacency matrix 초기화
dic_array = np.zeros((len(train),len(train)))
print(dic_array.shape)
print(dic_array.sum())

(5277, 5277)
0.0


In [ ]:
# 원하는 연결 구조에 따라 adjacency matrix에 값 추가
for i, (i_claim, i_m) in enumerate(zip(train.claim, train.published_date.dt.month)):
  for j, (j_claim, j_m) in enumerate(zip(train.claim, train.published_date.dt.month)):
    if i_m == j_m:
      dic_array[i][j] += 1

In [ ]:
# 완성된 Adjacency matrix 및 엣지 합
print(dic_array.shape)
print(dic_array.sum())

(5277, 5277)
2758845.0


In [ ]:
# feature 구축 예시 : random하게 node 당 dim 10의 feature 생성
feature = np.random.rand(dic_array.shape[0], 10).astype(np.float32)  # 데이터 유형을 float32로 설정

In [ ]:
# build graph 예시 : homogeneous graph

# build graph
edge_index = torch.nonzero(torch.tensor(dic_array), as_tuple=False).T
g = dgl.graph((edge_index[1], edge_index[0]))

# assign features
g.ndata['feature'] = torch.tensor(feature, dtype=torch.float32)
g.edata['weight'] = torch.tensor(coo_matrix(dic_array).data, dtype=torch.float32)

In [ ]:
coo_matrix(torch.tensor(dic_array)).data

array([1., 1., 1., ..., 1., 1., 1.])

In [ ]:
g

Graph(num_nodes=5277, num_edges=2758845,
      ndata_schemes={'feature': Scheme(shape=(10,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float32)})

이외에 학습을 위해 필요한 요소를 할당하여 그래프에 저장 후 학습을 수행하세요.
ex) labels, train_mask, valid_mask 등

In [ ]:
!pip install torch==2.2.1+cu121 torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.3/757.3 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
import torch
print("PyTorch has version {}".format(torch.__version__))
# install dgl
# https://www.dgl.ai/pages/start.html

PyTorch has version 2.2.1+cu121


In [ ]:
# Install DGL library
!pip install  dgl -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 51.4 MB/s eta 0:00:00


In [ ]:
# 필요한 라이브러리 설치
!pip install torch torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import torch
import dgl
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from dgl.nn.pytorch import GraphConv, GATConv
from torch.nn import Dropout, BatchNorm1d

# 데이터 로드
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Claim 텍스트 및 키워드 추출
claims = train['claim'].tolist()
keywords = train['keybert_keywords'].apply(eval).tolist()
test_claims = test['claim'].tolist()
test_keywords = test['keybert_keywords'].apply(eval).tolist()

# 키워드 간의 관련성 계산
def keyword_similarity(keyword_list):
    keyword_text = [' '.join(kw) for kw in keyword_list]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(keyword_text)
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return similarity_matrix

# 키워드 간의 관련성 계산
keyword_sim = keyword_similarity(keywords)

# 유사도 기반 그래프 생성
threshold = 0.5  # 유사도가 threshold 이상인 경우에만 엣지를 추가
edges = []
for i in range(len(keywords)):
    for j in range(i+1, len(keywords)):
        if keyword_sim[i][j] >= threshold:
            edges.append((i, j))

# 그래프 생성
g = dgl.graph(edges, num_nodes=len(train))

# self-loop 추가
g = dgl.add_self_loop(g)

# label 설정
labels = torch.tensor(train['label'].values, dtype=torch.long)

# Train/validation mask 생성
train_indices, val_indices = train_test_split(np.arange(len(labels)), test_size=0.2, stratify=labels)
train_mask = torch.tensor([i in train_indices for i in range(len(labels))], dtype=torch.bool)
val_mask = torch.tensor([i in val_indices for i in range(len(labels))], dtype=torch.bool)

# 그래프에 label 및 mask 추가
g.ndata['label'] = labels[:g.num_nodes()]
g.ndata['train_mask'] = train_mask
g.ndata['val_mask'] = val_mask

# Feature Engineering
combined_text = [' '.join([claim] + kw) for claim, kw in zip(claims, keywords)]
tfidf_vectorizer = TfidfVectorizer(max_features=2000)
tfidf_matrix = tfidf_vectorizer.fit_transform(combined_text).toarray()

# 추가 피처 생성 (예: claim 길이)
claim_lengths = np.array([len(claim.split()) for claim in claims]).reshape(-1, 1)
tfidf_matrix = np.hstack([tfidf_matrix, claim_lengths])

features = torch.tensor(tfidf_matrix, dtype=torch.float)
g.ndata['feat'] = features

# GAT 모델 정의 (Dropout 및 Batch Normalization 추가)
class GAT(torch.nn.Module):
    def __init__(self, in_feats, h_feats, num_classes, num_heads, dropout):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_feats, h_feats, num_heads)
        self.batch_norm1 = BatchNorm1d(h_feats * num_heads)
        self.conv2 = GATConv(h_feats * num_heads, h_feats, num_heads)
        self.batch_norm2 = BatchNorm1d(h_feats * num_heads)
        self.conv3 = GATConv(h_feats * num_heads, num_classes, num_heads)
        self.dropout = Dropout(dropout)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = h.view(h.shape[0], -1)
        h = self.batch_norm1(h)
        h = F.elu(h)
        h = self.dropout(h)
        h = self.conv2(g, h)
        h = h.view(h.shape[0], -1)
        h = self.batch_norm2(h)
        h = F.elu(h)
        h = self.dropout(h)
        h = self.conv3(g, h)
        h = h.mean(1)  # 여러 heads의 출력을 평균
        return h

# Hyperparameters
in_feats = features.shape[1]
h_feats = 256
num_heads = 8  # multi-head attention
num_classes = len(labels.unique())
learning_rate = 0.001
weight_decay = 1e-4
dropout = 0.3
num_epochs = 1000

# 모델 및 옵티마이저 초기화
model = GAT(in_feats, h_feats, num_classes, num_heads, dropout)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# 학습률 스케줄러 추가
from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5, verbose=True)

# Training 및 validation 함수 정의
def train(model, g, features, labels, train_mask, optimizer):
    model.train()
    logits = model(g, features)
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

# Early Stopping을 위한 설정
patience = 20
best_val_loss = float('inf')
patience_counter = 0

# Training 및 validation 루프
for epoch in range(num_epochs):
    loss = train(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['train_mask'], optimizer)
    train_acc = evaluate(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['train_mask'])
    val_acc = evaluate(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['val_mask'])
    val_loss = F.cross_entropy(model(g, g.ndata['feat'])[val_mask], g.ndata['label'][val_mask]).item()
    print(f"Epoch {epoch:05d} | Loss {loss:.4f} | Train Accuracy {train_acc:.4f} | Val Accuracy {val_acc:.4f} | Val Loss {val_loss:.4f}")

    # 학습률 스케줄러 스텝
    scheduler.step(val_loss)

    # Early Stopping 체크
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

# Test 데이터 처리
# Claim 텍스트와 키워드 결합 (학습 데이터와 동일한 방식)
test_combined_text = [' '.join([claim] + kw) for claim, kw in zip(test_claims, test_keywords)]
test_tfidf_matrix = tfidf_vectorizer.transform(test_combined_text).toarray()

# 추가 피처 생성 (예: claim 길이)
test_claim_lengths = np.array([len(claim.split()) for claim in test_claims]).reshape(-1, 1)
test_tfidf_matrix = np.hstack([test_tfidf_matrix, test_claim_lengths])

test_features = torch.tensor(test_tfidf_matrix, dtype=torch.float)

# 그래프에 feature 추가
test_g = dgl.graph([], num_nodes=len(test))
test_g = dgl.add_self_loop(test_g)
test_g.ndata['feat'] = test_features

# 모델을 평가 모드로 전환
model.eval()

# 테스트 데이터에 대한 예측 수행
with torch.no_grad():
    test_logits = model(test_g, test_g.ndata['feat'])
    _, test_pred = torch.max(test_logits, dim=1)

# 예측 결과를 DataFrame으로 변환
submission = pd.DataFrame({
    'ID': test['id'],
    'Label': test_pred.numpy()
})

# 예측 결과를 submission_trial_fin.csv 파일로 저장
submission.to_csv('submission_trial_fin______.csv', index=False)

print("Predictions saved to submission_trial_fin.csv")


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 00000 | Loss 0.7504 | Train Accuracy 0.4996 | Val Accuracy 0.4991 | Val Loss 3.8913
Epoch 00001 | Loss 3.5617 | Train Accuracy 0.5004 | Val Accuracy 0.5009 | Val Loss 1.2172
Epoch 00002 | Loss 1.0013 | Train Accuracy 0.5004 | Val Accuracy 0.5009 | Val Loss 1.0400
Epoch 00003 | Loss 0.6776 | Train Accuracy 0.5508 | Val Accuracy 0.5644 | Val Loss 0.6702
Epoch 00004 | Loss 0.6521 | Train Accuracy 0.5584 | Val Accuracy 0.5578 | Val Loss 0.6680
Epoch 00005 | Loss 0.6087 | Train Accuracy 0.5027 | Val Accuracy 0.5047 | Val Loss 0.7729
Epoch 00006 | Loss 0.4945 | Train Accuracy 0.5136 | Val Accuracy 0.5123 | Val Loss 0.9203
Epoch 00007 | Loss 0.5211 | Train Accuracy 0.5186 | Val Accuracy 0.5189 | Val Loss 0.8829
Epoch 00008 | Loss 0.5192 | Train Accuracy 0.5361 | Val Accuracy 0.5322 | Val Loss 0.7460
Epoch 00009 | Loss 0.4490 | Train Accuracy 0.6321 | Val Accuracy 0.5881 | Val Loss 0.6581
Epoch 00010 | Loss 0.4219 | Train Accuracy 0.7728 | Val Accuracy 0.6496 | Val Loss 0.6398
Epoch 0001

In [ ]:
import pandas as pd
import numpy as np
import torch
import dgl
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from dgl.nn.pytorch import GraphConv, GATConv
from torch.nn import Dropout, BatchNorm1d

# 데이터 로드
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Claim 텍스트 및 키워드 추출
claims = train['claim'].tolist()
keywords = train['keybert_keywords'].apply(eval).tolist()
test_claims = test['claim'].tolist()
test_keywords = test['keybert_keywords'].apply(eval).tolist()

# 키워드 간의 관련성 계산
def keyword_similarity(keyword_list):
    keyword_text = [' '.join(kw) for kw in keyword_list]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(keyword_text)
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return similarity_matrix

# 키워드 간의 관련성 계산
keyword_sim = keyword_similarity(keywords)

# 유사도 기반 그래프 생성
threshold = 0.5  # 유사도가 threshold 이상인 경우에만 엣지를 추가
edges = []
for i in range(len(keywords)):
    for j in range(i+1, len(keywords)):
        if keyword_sim[i][j] >= threshold:
            edges.append((i, j))

# 그래프 생성
g = dgl.graph(edges, num_nodes=len(train))

# self-loop 추가
g = dgl.add_self_loop(g)

# label 설정
labels = torch.tensor(train['label'].values, dtype=torch.long)

# Train/validation mask 생성
train_indices, val_indices = train_test_split(np.arange(len(labels)), test_size=0.2, stratify=labels)
train_mask = torch.tensor([i in train_indices for i in range(len(labels))], dtype=torch.bool)
val_mask = torch.tensor([i in val_indices for i in range(len(labels))], dtype=torch.bool)

# 그래프에 label 및 mask 추가
g.ndata['label'] = labels[:g.num_nodes()]
g.ndata['train_mask'] = train_mask
g.ndata['val_mask'] = val_mask

# Feature Engineering
combined_text = [' '.join([claim] + kw) for claim, kw in zip(claims, keywords)]
tfidf_vectorizer = TfidfVectorizer(max_features=2000)
tfidf_matrix = tfidf_vectorizer.fit_transform(combined_text).toarray()

# 추가 피처 생성 (예: claim 길이)
claim_lengths = np.array([len(claim.split()) for claim in claims]).reshape(-1, 1)
tfidf_matrix = np.hstack([tfidf_matrix, claim_lengths])

features = torch.tensor(tfidf_matrix, dtype=torch.float)
g.ndata['feat'] = features

# GAT 모델 정의 (Dropout 및 Batch Normalization 추가)
class GAT(torch.nn.Module):
    def __init__(self, in_feats, h_feats, num_classes, num_heads, dropout):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_feats, h_feats, num_heads)
        self.batch_norm1 = BatchNorm1d(h_feats * num_heads)
        self.conv2 = GATConv(h_feats * num_heads, h_feats, num_heads)
        self.batch_norm2 = BatchNorm1d(h_feats * num_heads)
        self.conv3 = GATConv(h_feats * num_heads, num_classes, num_heads)
        self.dropout = Dropout(dropout)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = h.view(h.shape[0], -1)
        h = self.batch_norm1(h)
        h = F.elu(h)
        h = self.dropout(h)
        h = self.conv2(g, h)
        h = h.view(h.shape[0], -1)
        h = self.batch_norm2(h)
        h = F.elu(h)
        h = self.dropout(h)
        h = self.conv3(g, h)
        h = h.mean(1)  # 여러 heads의 출력을 평균
        return h

# Hyperparameters
in_feats = features.shape[1]
h_feats = 256
num_heads = 8  # multi-head attention
num_classes = len(labels.unique())
learning_rate = 0.0001
weight_decay = 1e-4
dropout = 0.3
num_epochs = 1000

# 모델 및 옵티마이저 초기화
model = GAT(in_feats, h_feats, num_classes, num_heads, dropout)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# 학습률 스케줄러 추가
from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5, verbose=True)

# Training 및 validation 함수 정의
def train(model, g, features, labels, train_mask, optimizer):
    model.train()
    logits = model(g, features)
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

# Early Stopping을 위한 설정
patience = 20
best_val_loss = float('inf')
patience_counter = 0

# Training 및 validation 루프
for epoch in range(num_epochs):
    loss = train(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['train_mask'], optimizer)
    train_acc = evaluate(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['train_mask'])
    val_acc = evaluate(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['val_mask'])
    val_loss = F.cross_entropy(model(g, g.ndata['feat'])[val_mask], g.ndata['label'][val_mask]).item()
    print(f"Epoch {epoch:05d} | Loss {loss:.4f} | Train Accuracy {train_acc:.4f} | Val Accuracy {val_acc:.4f} | Val Loss {val_loss:.4f}")

    # 학습률 스케줄러 스텝
    scheduler.step(val_loss)

    # Early Stopping 체크
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

# Test 데이터 처리
# Claim 텍스트와 키워드 결합 (학습 데이터와 동일한 방식)
test_combined_text = [' '.join([claim] + kw) for claim, kw in zip(test_claims, test_keywords)]
test_tfidf_matrix = tfidf_vectorizer.transform(test_combined_text).toarray()

# 추가 피처 생성 (예: claim 길이)
test_claim_lengths = np.array([len(claim.split()) for claim in test_claims]).reshape(-1, 1)
test_tfidf_matrix = np.hstack([test_tfidf_matrix, test_claim_lengths])

test_features = torch.tensor(test_tfidf_matrix, dtype=torch.float)

# 그래프에 feature 추가
test_g = dgl.graph([], num_nodes=len(test))
test_g = dgl.add_self_loop(test_g)
test_g.ndata['feat'] = test_features

# 모델을 평가 모드로 전환
model.eval()

# 테스트 데이터에 대한 예측 수행
with torch.no_grad():
    test_logits = model(test_g, test_g.ndata['feat'])
    _, test_pred = torch.max(test_logits, dim=1)

# 예측 결과를 DataFrame으로 변환
submission = pd.DataFrame({
    'ID': test['id'],
    'Label': test_pred.numpy()
})

# 예측 결과를 submission_trial_fin.csv 파일로 저장
submission.to_csv('submission_trial_fin_____444_.csv', index=False)

print("Predictions saved to submission_trial_fin.csv")


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 00000 | Loss 0.8027 | Train Accuracy 0.4996 | Val Accuracy 0.4991 | Val Loss 0.8250
Epoch 00001 | Loss 0.9755 | Train Accuracy 0.4975 | Val Accuracy 0.5000 | Val Loss 0.6933
Epoch 00002 | Loss 0.7510 | Train Accuracy 0.5001 | Val Accuracy 0.5019 | Val Loss 0.7559
Epoch 00003 | Loss 0.7722 | Train Accuracy 0.5015 | Val Accuracy 0.5009 | Val Loss 0.7908
Epoch 00004 | Loss 0.8202 | Train Accuracy 0.5015 | Val Accuracy 0.5000 | Val Loss 0.7424
Epoch 00005 | Loss 0.7288 | Train Accuracy 0.5023 | Val Accuracy 0.5009 | Val Loss 0.6963
Epoch 00006 | Loss 0.6734 | Train Accuracy 0.5631 | Val Accuracy 0.5720 | Val Loss 0.6813
Epoch 00007 | Loss 0.6810 | Train Accuracy 0.5508 | Val Accuracy 0.5331 | Val Loss 0.6806
Epoch 00008 | Loss 0.6888 | Train Accuracy 0.5620 | Val Accuracy 0.5578 | Val Loss 0.6788
Epoch 00009 | Loss 0.6815 | Train Accuracy 0.5314 | Val Accuracy 0.5341 | Val Loss 0.6815
Epoch 00010 | Loss 0.6478 | Train Accuracy 0.5030 | Val Accuracy 0.5019 | Val Loss 0.6982
Epoch 0001

In [ ]:
import pandas as pd
import numpy as np
import torch
import dgl
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from dgl.nn.pytorch import GATConv
from torch.nn import Dropout, BatchNorm1d
from textblob import TextBlob

# 데이터 로드
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Claim 텍스트 및 키워드 추출
claims = train['claim'].tolist()
keywords = train['keybert_keywords'].apply(eval).tolist()
test_claims = test['claim'].tolist()
test_keywords = test['keybert_keywords'].apply(eval).tolist()

# 키워드 간의 관련성 계산
def keyword_similarity(keyword_list):
    keyword_text = [' '.join(kw) for kw in keyword_list]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(keyword_text)
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return similarity_matrix

# 키워드 간의 관련성 계산
keyword_sim = keyword_similarity(keywords)

# 유사도 기반 그래프 생성
threshold = 0.5  # 유사도가 threshold 이상인 경우에만 엣지를 추가
edges = []
for i in range(len(keywords)):
    for j in range(i + 1, len(keywords)):
        if keyword_sim[i][j] >= threshold:
            edges.append((i, j))

# 그래프 생성
g = dgl.graph(edges, num_nodes=len(train))

# self-loop 추가
g = dgl.add_self_loop(g)

# label 설정
labels = torch.tensor(train['label'].values, dtype=torch.long)

# Train/validation mask 생성
train_indices, val_indices = train_test_split(np.arange(len(labels)), test_size=0.2, stratify=labels)
train_mask = torch.tensor([i in train_indices for i in range(len(labels))], dtype=torch.bool)
val_mask = torch.tensor([i in val_indices for i in range(len(labels))], dtype=torch.bool)

# 그래프에 label 및 mask 추가
g.ndata['label'] = labels[:g.num_nodes()]
g.ndata['train_mask'] = train_mask
g.ndata['val_mask'] = val_mask

# Feature Engineering
combined_text = [' '.join([claim] + kw) for claim, kw in zip(claims, keywords)]
tfidf_vectorizer = TfidfVectorizer(max_features=2000)
tfidf_matrix = tfidf_vectorizer.fit_transform(combined_text).toarray()

# 추가 피처 생성 (예: claim 길이)
claim_lengths = np.array([len(claim.split()) for claim in claims]).reshape(-1, 1)
tfidf_matrix = np.hstack([tfidf_matrix, claim_lengths])

# 감정 점수 추가
sentiments = np.array([TextBlob(claim).sentiment.polarity for claim in claims]).reshape(-1, 1)
tfidf_matrix = np.hstack([tfidf_matrix, sentiments])

features = torch.tensor(tfidf_matrix, dtype=torch.float)
g.ndata['feat'] = features

# GAT 모델 정의 (Dropout 및 Batch Normalization 추가)
class GATImproved(torch.nn.Module):
    def __init__(self, in_feats, h_feats, num_classes, num_heads, dropout):
        super(GATImproved, self).__init__()
        self.conv1 = GATConv(in_feats, h_feats, num_heads, allow_zero_in_degree=True)
        self.batch_norm1 = BatchNorm1d(h_feats * num_heads)
        self.conv2 = GATConv(h_feats * num_heads, h_feats, num_heads, allow_zero_in_degree=True)
        self.batch_norm2 = BatchNorm1d(h_feats * num_heads)
        self.conv3 = GATConv(h_feats * num_heads, h_feats, num_heads, allow_zero_in_degree=True)
        self.batch_norm3 = BatchNorm1d(h_feats * num_heads)
        self.conv4 = GATConv(h_feats * num_heads, num_classes, num_heads, allow_zero_in_degree=True)
        self.dropout = Dropout(dropout)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = h.view(h.shape[0], -1)
        h = self.batch_norm1(h)
        h = F.elu(h)
        h = self.dropout(h)
        h = self.conv2(g, h)
        h = h.view(h.shape[0], -1)
        h = self.batch_norm2(h)
        h = F.elu(h)
        h = self.dropout(h)
        h = self.conv3(g, h)
        h = h.view(h.shape[0], -1)
        h = self.batch_norm3(h)
        h = F.elu(h)
        h = self.dropout(h)
        h = self.conv4(g, h)
        h = h.mean(1)  # 여러 heads의 출력을 평균
        return h

# Hyperparameters
in_feats = features.shape[1]
h_feats = 512  # hidden feature 수를 늘림
num_heads = 16  # attention head 수를 늘림
num_classes = len(labels.unique())
learning_rate = 0.0001
weight_decay = 1e-4
dropout = 0.4  # dropout 비율을 높임
num_epochs = 1000

# 모델 및 옵티마이저 초기화
model = GATImproved(in_feats, h_feats, num_classes, num_heads, dropout)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# 학습률 스케줄러 추가
from torch.optim.lr_scheduler import StepLR
scheduler = StepLR(optimizer, step_size=50, gamma=0.5)

# Training 및 validation 함수 정의
def train(model, g, features, labels, train_mask, optimizer):
    model.train()
    logits = model(g, features)
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

# Early Stopping을 위한 설정
patience = 20
best_val_loss = float('inf')
patience_counter = 0

# Training 및 validation 루프
for epoch in range(num_epochs):
    loss = train(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['train_mask'], optimizer)
    train_acc = evaluate(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['train_mask'])
    val_acc = evaluate(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['val_mask'])
    val_loss = F.cross_entropy(model(g, g.ndata['feat'])[g.ndata['val_mask']], g.ndata['label'][g.ndata['val_mask']]).item()
    print(f"Epoch {epoch:05d} | Loss {loss:.4f} | Train Accuracy {train_acc:.4f} | Val Accuracy {val_acc:.4f} | Val Loss {val_loss:.4f}")

    # 학습률 스케줄러 스텝
    scheduler.step()

    # Early Stopping 체크
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

# Test 데이터 처리
test_combined_text = [' '.join([claim] + kw) for claim, kw in zip(test_claims, test_keywords)]
test_tfidf_matrix = tfidf_vectorizer.transform(test_combined_text).toarray()

# 추가 피처 생성 (예: claim 길이)
test_claim_lengths = np.array([len(claim.split()) for claim in test_claims]).reshape(-1, 1)
test_tfidf_matrix = np.hstack([test_tfidf_matrix, test_claim_lengths])

# 감정 점수 추가
test_sentiments = np.array([TextBlob(claim).sentiment.polarity for claim in test_claims]).reshape(-1, 1)
test_tfidf_matrix = np.hstack([test_tfidf_matrix, test_sentiments])

test_features = torch.tensor(test_tfidf_matrix, dtype=torch.float)

# 테스트 그래프 생성 및 특성 추가
test_g = dgl.graph([], num_nodes=len(test))
test_g = dgl.add_self_loop(test_g)
test_g.ndata['feat'] = test_features

# 모델을 평가 모드로 전환
model.eval()

# 테스트 데이터에 대한 예측 수행
with torch.no_grad():
    test_logits = model(test_g, test_g.ndata['feat'])
    _, test_pred = torch.max(test_logits, dim=1)

# 예측 결과를 DataFrame으로 변환
submission = pd.DataFrame({
    'ID': test['id'],
    'Label': test_pred.numpy()
})

# 예측 결과를 CSV 파일로 저장
submission.to_csv('submission_improved.csv', index=False)

print("Predictions saved to submission_improved.csv")


Epoch 00000 | Loss 0.7419 | Train Accuracy 0.4996 | Val Accuracy 0.4991 | Val Loss 1.0326
Epoch 00001 | Loss 1.1047 | Train Accuracy 0.5004 | Val Accuracy 0.5009 | Val Loss 3.2851
Epoch 00002 | Loss 2.7782 | Train Accuracy 0.5004 | Val Accuracy 0.5009 | Val Loss 1.3203
Epoch 00003 | Loss 1.1023 | Train Accuracy 0.4921 | Val Accuracy 0.4915 | Val Loss 1.2760
Epoch 00004 | Loss 1.5442 | Train Accuracy 0.4932 | Val Accuracy 0.4915 | Val Loss 1.2830
Epoch 00005 | Loss 1.5820 | Train Accuracy 0.5108 | Val Accuracy 0.4962 | Val Loss 0.7261
Epoch 00006 | Loss 0.7691 | Train Accuracy 0.5103 | Val Accuracy 0.5095 | Val Loss 0.9888
Epoch 00007 | Loss 0.9780 | Train Accuracy 0.5091 | Val Accuracy 0.5114 | Val Loss 1.1456
Epoch 00008 | Loss 1.1862 | Train Accuracy 0.5110 | Val Accuracy 0.5133 | Val Loss 0.9243
Epoch 00009 | Loss 0.8458 | Train Accuracy 0.5070 | Val Accuracy 0.5133 | Val Loss 0.6952
Epoch 00010 | Loss 0.6138 | Train Accuracy 0.5167 | Val Accuracy 0.5057 | Val Loss 0.7227
Epoch 0001

In [ ]:
import pandas as pd
import numpy as np
import torch
import dgl
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from dgl.nn.pytorch import GATConv
from torch.nn import Dropout, BatchNorm1d
from textblob import TextBlob

# 데이터 로드
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Claim 텍스트 및 키워드 추출
claims = train['claim'].tolist()
keywords = train['keybert_keywords'].apply(eval).tolist()
test_claims = test['claim'].tolist()
test_keywords = test['keybert_keywords'].apply(eval).tolist()

# 키워드 간의 관련성 계산
def keyword_similarity(keyword_list):
    keyword_text = [' '.join(kw) for kw in keyword_list]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(keyword_text)
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return similarity_matrix

# 키워드 간의 관련성 계산
keyword_sim = keyword_similarity(keywords)

# 유사도 기반 그래프 생성
threshold = 0.5  # 유사도가 threshold 이상인 경우에만 엣지를 추가
edges = []
for i in range(len(keywords)):
    for j in range(i + 1, len(keywords)):
        if keyword_sim[i][j] >= threshold:
            edges.append((i, j))

# 그래프 생성
g = dgl.graph(edges, num_nodes=len(train))

# self-loop 추가
g = dgl.add_self_loop(g)

# label 설정
labels = torch.tensor(train['label'].values, dtype=torch.long)

# Train/validation mask 생성
train_indices, val_indices = train_test_split(np.arange(len(labels)), test_size=0.2, stratify=labels)
train_mask = torch.tensor([i in train_indices for i in range(len(labels))], dtype=torch.bool)
val_mask = torch.tensor([i in val_indices for i in range(len(labels))], dtype=torch.bool)

# 그래프에 label 및 mask 추가
g.ndata['label'] = labels[:g.num_nodes()]
g.ndata['train_mask'] = train_mask
g.ndata['val_mask'] = val_mask

# Feature Engineering
combined_text = [' '.join([claim] + kw) for claim, kw in zip(claims, keywords)]
tfidf_vectorizer = TfidfVectorizer(max_features=2000)
tfidf_matrix = tfidf_vectorizer.fit_transform(combined_text).toarray()

# 추가 피처 생성 (예: claim 길이)
claim_lengths = np.array([len(claim.split()) for claim in claims]).reshape(-1, 1)
tfidf_matrix = np.hstack([tfidf_matrix, claim_lengths])

# 감정 점수 추가
sentiments = np.array([TextBlob(claim).sentiment.polarity for claim in claims]).reshape(-1, 1)
tfidf_matrix = np.hstack([tfidf_matrix, sentiments])

features = torch.tensor(tfidf_matrix, dtype=torch.float)
g.ndata['feat'] = features

# GAT 모델 정의 (Dropout 및 Batch Normalization 추가)
class GATImproved(torch.nn.Module):
    def __init__(self, in_feats, h_feats, num_classes, num_heads, dropout):
        super(GATImproved, self).__init__()
        self.conv1 = GATConv(in_feats, h_feats, num_heads, allow_zero_in_degree=True)
        self.batch_norm1 = BatchNorm1d(h_feats * num_heads)
        self.conv2 = GATConv(h_feats * num_heads, h_feats, num_heads, allow_zero_in_degree=True)
        self.batch_norm2 = BatchNorm1d(h_feats * num_heads)
        self.conv3 = GATConv(h_feats * num_heads, h_feats, num_heads, allow_zero_in_degree=True)
        self.batch_norm3 = BatchNorm1d(h_feats * num_heads)
        self.conv4 = GATConv(h_feats * num_heads, num_classes, num_heads, allow_zero_in_degree=True)
        self.dropout = Dropout(dropout)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = h.view(h.shape[0], -1)
        h = self.batch_norm1(h)
        h = F.elu(h)
        h = self.dropout(h)
        h = self.conv2(g, h)
        h = h.view(h.shape[0], -1)
        h = self.batch_norm2(h)
        h = F.elu(h)
        h = self.dropout(h)
        h = self.conv3(g, h)
        h = h.view(h.shape[0], -1)
        h = self.batch_norm3(h)
        h = F.elu(h)
        h = self.dropout(h)
        h = self.conv4(g, h)
        h = h.mean(1)  # 여러 heads의 출력을 평균
        return h

# Hyperparameters
in_feats = features.shape[1]
h_feats = 512  # hidden feature 수를 늘림
num_heads = 16  # attention head 수를 늘림
num_classes = len(labels.unique())
learning_rate = 0.001
weight_decay = 1e-4
dropout = 0.4  # dropout 비율을 높임
num_epochs = 1000

# 모델 및 옵티마이저 초기화
model = GATImproved(in_feats, h_feats, num_classes, num_heads, dropout)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# 학습률 스케줄러 추가
from torch.optim.lr_scheduler import StepLR
scheduler = StepLR(optimizer, step_size=50, gamma=0.5)

# Training 및 validation 함수 정의
def train(model, g, features, labels, train_mask, optimizer):
    model.train()
    logits = model(g, features)
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

# Early Stopping을 위한 설정
patience = 20
best_val_loss = float('inf')
patience_counter = 0

# Training 및 validation 루프
for epoch in range(num_epochs):
    loss = train(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['train_mask'], optimizer)
    train_acc = evaluate(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['train_mask'])
    val_acc = evaluate(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['val_mask'])
    val_loss = F.cross_entropy(model(g, g.ndata['feat'])[g.ndata['val_mask']], g.ndata['label'][g.ndata['val_mask']]).item()
    print(f"Epoch {epoch:05d} | Loss {loss:.4f} | Train Accuracy {train_acc:.4f} | Val Accuracy {val_acc:.4f} | Val Loss {val_loss:.4f}")

    # 학습률 스케줄러 스텝
    scheduler.step()

    # Early Stopping 체크
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

# Test 데이터 처리
test_combined_text = [' '.join([claim] + kw) for claim, kw in zip(test_claims, test_keywords)]
test_tfidf_matrix = tfidf_vectorizer.transform(test_combined_text).toarray()

# 추가 피처 생성 (예: claim 길이)
test_claim_lengths = np.array([len(claim.split()) for claim in test_claims]).reshape(-1, 1)
test_tfidf_matrix = np.hstack([test_tfidf_matrix, test_claim_lengths])

# 감정 점수 추가
test_sentiments = np.array([TextBlob(claim).sentiment.polarity for claim in test_claims]).reshape(-1, 1)
test_tfidf_matrix = np.hstack([test_tfidf_matrix, test_sentiments])

test_features = torch.tensor(test_tfidf_matrix, dtype=torch.float)

# 테스트 그래프 생성 및 특성 추가
test_g = dgl.graph([], num_nodes=len(test))
test_g = dgl.add_self_loop(test_g)
test_g.ndata['feat'] = test_features

# 모델을 평가 모드로 전환
model.eval()

# 테스트 데이터에 대한 예측 수행
with torch.no_grad():
    test_logits = model(test_g, test_g.ndata['feat'])
    _, test_pred = torch.max(test_logits, dim=1)

# 예측 결과를 DataFrame으로 변환
submission = pd.DataFrame({
    'ID': test['id'],
    'Label': test_pred.numpy()
})

# 예측 결과를 CSV 파일로 저장
submission.to_csv('submission_improved.csv', index=False)

print("Predictions saved to submission_improved_2.csv")

Epoch 00000 | Loss 0.7528 | Train Accuracy 0.4899 | Val Accuracy 0.5057 | Val Loss 5.5942
Epoch 00001 | Loss 5.5178 | Train Accuracy 0.4994 | Val Accuracy 0.5000 | Val Loss 27.6033
Epoch 00002 | Loss 8.5856 | Train Accuracy 0.4996 | Val Accuracy 0.4991 | Val Loss 3.9894
Epoch 00003 | Loss 1.5533 | Train Accuracy 0.5049 | Val Accuracy 0.5028 | Val Loss 7.1989
Epoch 00004 | Loss 2.5966 | Train Accuracy 0.5079 | Val Accuracy 0.5038 | Val Loss 4.8727
Epoch 00005 | Loss 2.1869 | Train Accuracy 0.5203 | Val Accuracy 0.5133 | Val Loss 1.1887
Epoch 00006 | Loss 0.9590 | Train Accuracy 0.4911 | Val Accuracy 0.5028 | Val Loss 1.9313
Epoch 00007 | Loss 1.2426 | Train Accuracy 0.4914 | Val Accuracy 0.5000 | Val Loss 1.9350
Epoch 00008 | Loss 1.4473 | Train Accuracy 0.4800 | Val Accuracy 0.4650 | Val Loss 1.0647
Epoch 00009 | Loss 0.8646 | Train Accuracy 0.5200 | Val Accuracy 0.5009 | Val Loss 0.8239
Epoch 00010 | Loss 0.7253 | Train Accuracy 0.5217 | Val Accuracy 0.5038 | Val Loss 1.1227
Epoch 000

In [ ]:
import pandas as pd
import numpy as np
import torch
import dgl
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from dgl.nn.pytorch import GATConv
from torch.nn import Dropout, BatchNorm1d
from textblob import TextBlob
from transformers import BertTokenizer, BertModel

# 데이터 로드
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Claim 텍스트 및 키워드 추출
claims = train['claim'].tolist()
keywords = train['keybert_keywords'].apply(eval).tolist()
test_claims = test['claim'].tolist()
test_keywords = test['keybert_keywords'].apply(eval).tolist()

# 키워드 간의 관련성 계산
def keyword_similarity(keyword_list):
    keyword_text = [' '.join(kw) for kw in keyword_list]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(keyword_text)
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return similarity_matrix

# 키워드 간의 관련성 계산
keyword_sim = keyword_similarity(keywords)

# 유사도 기반 그래프 생성
threshold = 0.5  # 유사도가 threshold 이상인 경우에만 엣지를 추가
edges = []
for i in range(len(keywords)):
    for j in range(i + 1, len(keywords)):
        if keyword_sim[i][j] >= threshold:
            edges.append((i, j))

# 그래프 생성
g = dgl.graph(edges, num_nodes=len(train))

# self-loop 추가
g = dgl.add_self_loop(g)

# label 설정
labels = torch.tensor(train['label'].values, dtype=torch.long)

# Train/validation mask 생성
train_indices, val_indices = train_test_split(np.arange(len(labels)), test_size=0.2, stratify=labels)
train_mask = torch.tensor([i in train_indices for i in range(len(labels))], dtype=torch.bool)
val_mask = torch.tensor([i in val_indices for i in range(len(labels))], dtype=torch.bool)

# 그래프에 label 및 mask 추가
g.ndata['label'] = labels[:g.num_nodes()]
g.ndata['train_mask'] = train_mask
g.ndata['val_mask'] = val_mask

# Feature Engineering
# BERT 임베딩 사용
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embedding(text_list):
    inputs = tokenizer(text_list, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    return embeddings

claim_embeddings = get_bert_embedding(claims)
keyword_embeddings = get_bert_embedding([' '.join(kw) for kw in keywords])

# Combine embeddings
combined_embeddings = np.concatenate((claim_embeddings, keyword_embeddings), axis=1)

# 추가 피처 생성 (예: claim 길이)
claim_lengths = np.array([len(claim.split()) for claim in claims]).reshape(-1, 1)
combined_embeddings = np.hstack([combined_embeddings, claim_lengths])

# 감정 점수 추가
sentiments = np.array([TextBlob(claim).sentiment.polarity for claim in claims]).reshape(-1, 1)
combined_embeddings = np.hstack([combined_embeddings, sentiments])

features = torch.tensor(combined_embeddings, dtype=torch.float)
g.ndata['feat'] = features

# GAT 모델 정의 (Dropout 및 Batch Normalization 추가)
class GATImproved(torch.nn.Module):
    def __init__(self, in_feats, h_feats, num_classes, num_heads, dropout):
        super(GATImproved, self).__init__()
        self.conv1 = GATConv(in_feats, h_feats, num_heads, allow_zero_in_degree=True)
        self.batch_norm1 = BatchNorm1d(h_feats * num_heads)
        self.conv2 = GATConv(h_feats * num_heads, h_feats, num_heads, allow_zero_in_degree=True)
        self.batch_norm2 = BatchNorm1d(h_feats * num_heads)
        self.conv3 = GATConv(h_feats * num_heads, h_feats, num_heads, allow_zero_in_degree=True)
        self.batch_norm3 = BatchNorm1d(h_feats * num_heads)
        self.conv4 = GATConv(h_feats * num_heads, num_classes, num_heads, allow_zero_in_degree=True)
        self.dropout = Dropout(dropout)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = h.view(h.shape[0], -1)
        h = self.batch_norm1(h)
        h = F.elu(h)
        h = self.dropout(h)
        h = self.conv2(g, h)
        h = h.view(h.shape[0], -1)
        h = self.batch_norm2(h)
        h = F.elu(h)
        h = self.dropout(h)
        h = self.conv3(g, h)
        h = h.view(h.shape[0], -1)
        h = self.batch_norm3(h)
        h = F.elu(h)
        h = self.dropout(h)
        h = self.conv4(g, h)
        h = h.mean(1)  # 여러 heads의 출력을 평균
        return h

# Hyperparameters
in_feats = features.shape[1]
h_feats = 512  # hidden feature 수를 늘림
num_heads = 16  # attention head 수를 늘림
num_classes = len(labels.unique())
learning_rate = 0.001
weight_decay = 1e-4
dropout = 0.4  # dropout 비율을 높임
num_epochs = 1000

# 모델 및 옵티마이저 초기화
model = GATImproved(in_feats, h_feats, num_classes, num_heads, dropout)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# 학습률 스케줄러 추가
from torch.optim.lr_scheduler import StepLR
scheduler = StepLR(optimizer, step_size=50, gamma=0.5)

# Training 및 validation 함수 정의
def train(model, g, features, labels, train_mask, optimizer):
    model.train()
    logits = model(g, features)
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

# Early Stopping을 위한 설정
patience = 20
best_val_loss = float('inf')
patience_counter = 0

# Training 및 validation 루프
for epoch in range(num_epochs):
    loss = train(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['train_mask'], optimizer)
    train_acc = evaluate(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['train_mask'])
    val_acc = evaluate(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['val_mask'])
    val_loss = F.cross_entropy(model(g, g.ndata['feat'])[g.ndata['val_mask']], g.ndata['label'][g.ndata['val_mask']]).item()
    print(f"Epoch {epoch:05d} | Loss {loss:.4f} | Train Accuracy {train_acc:.4f} | Val Accuracy {val_acc:.4f} | Val Loss {val_loss:.4f}")

    # 학습률 스케줄러 스텝
    scheduler.step()

    # Early Stopping 체크
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

# Test 데이터 처리
test_combined_text = [' '.join([claim] + kw) for claim, kw in zip(test_claims, test_keywords)]
test_claim_embeddings = get_bert_embedding(test_claims)
test_keyword_embeddings = get_bert_embedding([' '.join(kw) for kw in test_keywords])

# Combine embeddings
test_combined_embeddings = np.concatenate((test_claim_embeddings, test_keyword_embeddings), axis=1)

# 추가 피처 생성 (예: claim 길이)
test_claim_lengths = np.array([len(claim.split()) for claim in test_claims]).reshape(-1, 1)
test_combined_embeddings = np.hstack([test_combined_embeddings, test_claim_lengths])

# 감정 점수 추가
test_sentiments = np.array([TextBlob(claim).sentiment.polarity for claim in test_claims]).reshape(-1, 1)
test_combined_embeddings = np.hstack([test_combined_embeddings, test_sentiments])

test_features = torch.tensor(test_combined_embeddings, dtype=torch.float)

# 테스트 그래프 생성 및 특성 추가
test_g = dgl.graph([], num_nodes=len(test))
test_g = dgl.add_self_loop(test_g)
test_g.ndata['feat'] = test_features

# 모델을 평가 모드로 전환
model.eval()

# 테스트 데이터에 대한 예측 수행
with torch.no_grad():
    test_logits = model(test_g, test_g.ndata['feat'])
    _, test_pred = torch.max(test_logits, dim=1)

# 예측 결과를 DataFrame으로 변환
submission = pd.DataFrame({
    'ID': test['id'],
    'Label': test_pred.numpy()
})

# 예측 결과를 CSV 파일로 저장
submission.to_csv('submission_improved__.csv', index=False)

print("Predictions saved to submission_improved.csv")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch 00000 | Loss 0.7489 | Train Accuracy 0.6100 | Val Accuracy 0.5900 | Val Loss 5.1662
Epoch 00001 | Loss 4.1011 | Train Accuracy 0.4385 | Val Accuracy 0.4517 | Val Loss 25.0884
Epoch 00002 | Loss 15.5015 | Train Accuracy 0.4996 | Val Accuracy 0.4991 | Val Loss 14.5947
Epoch 00003 | Loss 6.1387 | Train Accuracy 0.5577 | Val Accuracy 0.5616 | Val Loss 2.6350
Epoch 00004 | Loss 1.5916 | Train Accuracy 0.4425 | Val Accuracy 0.4403 | Val Loss 4.7826
Epoch 00005 | Loss 1.9913 | Train Accuracy 0.4250 | Val Accuracy 0.4195 | Val Loss 6.9193
Epoch 00006 | Loss 2.9633 | Train Accuracy 0.4160 | Val Accuracy 0.4261 | Val Loss 5.3029
Epoch 00007 | Loss 2.1051 | Train Accuracy 0.4385 | Val Accuracy 0.4356 | Val Loss 2.2975
Epoch 00008 | Loss 0.7714 | Train Accuracy 0.6487 | Val Accuracy 0.6354 | Val Loss 1.0151
Epoch 00009 | Loss 0.9773 | Train Accuracy 0.6259 | Val Accuracy 0.6023 | Val Loss 1.6287
Epoch 00010 | Loss 1.4018 | Train Accuracy 0.6195 | Val Accuracy 0.6023 | Val Loss 1.9124
Epoch 0

In [ ]:
import pandas as pd
import numpy as np
import torch
import dgl
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from dgl.nn.pytorch import GATConv
from torch.nn import Dropout, BatchNorm1d
from textblob import TextBlob
from transformers import BertTokenizer, BertModel

# 데이터 로드
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Claim 텍스트 및 키워드 추출
claims = train['claim'].tolist()
keywords = train['keybert_keywords'].apply(eval).tolist()
test_claims = test['claim'].tolist()
test_keywords = test['keybert_keywords'].apply(eval).tolist()

# 키워드 간의 관련성 계산
def keyword_similarity(keyword_list):
    keyword_text = [' '.join(kw) for kw in keyword_list]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(keyword_text)
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return similarity_matrix

# 키워드 간의 관련성 계산
keyword_sim = keyword_similarity(keywords)

# 유사도 기반 그래프 생성
threshold = 0.5  # 유사도가 threshold 이상인 경우에만 엣지를 추가
edges = []
for i in range(len(keywords)):
    for j in range(i + 1, len(keywords)):
        if keyword_sim[i][j] >= threshold:
            edges.append((i, j))

# 그래프 생성
g = dgl.graph(edges, num_nodes=len(train))

# self-loop 추가
g = dgl.add_self_loop(g)

# label 설정
labels = torch.tensor(train['label'].values, dtype=torch.long)

# Train/validation mask 생성
train_indices, val_indices = train_test_split(np.arange(len(labels)), test_size=0.2, stratify=labels)
train_mask = torch.tensor([i in train_indices for i in range(len(labels))], dtype=torch.bool)
val_mask = torch.tensor([i in val_indices for i in range(len(labels))], dtype=torch.bool)

# 그래프에 label 및 mask 추가
g.ndata['label'] = labels[:g.num_nodes()]
g.ndata['train_mask'] = train_mask
g.ndata['val_mask'] = val_mask

# Feature Engineering
# BERT 임베딩 사용
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embedding(text_list):
    inputs = tokenizer(text_list, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    return embeddings

claim_embeddings = get_bert_embedding(claims)
keyword_embeddings = get_bert_embedding([' '.join(kw) for kw in keywords])

# Combine embeddings
combined_embeddings = np.concatenate((claim_embeddings, keyword_embeddings), axis=1)

# 추가 피처 생성 (예: claim 길이)
claim_lengths = np.array([len(claim.split()) for claim in claims]).reshape(-1, 1)
combined_embeddings = np.hstack([combined_embeddings, claim_lengths])

# 감정 점수 추가
sentiments = np.array([TextBlob(claim).sentiment.polarity for claim in claims]).reshape(-1, 1)
combined_embeddings = np.hstack([combined_embeddings, sentiments])

features = torch.tensor(combined_embeddings, dtype=torch.float)
g.ndata['feat'] = features

# GAT 모델 정의 (Dropout 및 Batch Normalization 추가)
class GATImproved(torch.nn.Module):
    def __init__(self, in_feats, h_feats, num_classes, num_heads, dropout):
        super(GATImproved, self).__init__()
        self.conv1 = GATConv(in_feats, h_feats, num_heads, allow_zero_in_degree=True)
        self.batch_norm1 = BatchNorm1d(h_feats * num_heads)
        self.conv2 = GATConv(h_feats * num_heads, h_feats, num_heads, allow_zero_in_degree=True)
        self.batch_norm2 = BatchNorm1d(h_feats * num_heads)
        self.conv3 = GATConv(h_feats * num_heads, h_feats, num_heads, allow_zero_in_degree=True)
        self.batch_norm3 = BatchNorm1d(h_feats * num_heads)
        self.conv4 = GATConv(h_feats * num_heads, num_classes, num_heads, allow_zero_in_degree=True)
        self.dropout = Dropout(dropout)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = h.view(h.shape[0], -1)
        h = self.batch_norm1(h)
        h = F.elu(h)
        h = self.dropout(h)
        h = self.conv2(g, h)
        h = h.view(h.shape[0], -1)
        h = self.batch_norm2(h)
        h = F.elu(h)
        h = self.dropout(h)
        h = self.conv3(g, h)
        h = h.view(h.shape[0], -1)
        h = self.batch_norm3(h)
        h = F.elu(h)
        h = self.dropout(h)
        h = self.conv4(g, h)
        h = h.mean(1)  # 여러 heads의 출력을 평균
        return h

# Hyperparameters
in_feats = features.shape[1]
h_feats = 512  # hidden feature 수를 늘림
num_heads = 16  # attention head 수를 늘림
num_classes = len(labels.unique())
learning_rate = 0.0001
weight_decay = 1e-4
dropout = 0.4  # dropout 비율을 높임
num_epochs = 1000

# 모델 및 옵티마이저 초기화
model = GATImproved(in_feats, h_feats, num_classes, num_heads, dropout)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# 학습률 스케줄러 추가
from torch.optim.lr_scheduler import StepLR
scheduler = StepLR(optimizer, step_size=50, gamma=0.5)

# Training 및 validation 함수 정의
def train(model, g, features, labels, train_mask, optimizer):
    model.train()
    logits = model(g, features)
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

# Early Stopping을 위한 설정
patience = 20
best_val_loss = float('inf')
patience_counter = 0

# Training 및 validation 루프
for epoch in range(num_epochs):
    loss = train(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['train_mask'], optimizer)
    train_acc = evaluate(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['train_mask'])
    val_acc = evaluate(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['val_mask'])
    val_loss = F.cross_entropy(model(g, g.ndata['feat'])[g.ndata['val_mask']], g.ndata['label'][g.ndata['val_mask']]).item()
    print(f"Epoch {epoch:05d} | Loss {loss:.4f} | Train Accuracy {train_acc:.4f} | Val Accuracy {val_acc:.4f} | Val Loss {val_loss:.4f}")

    # 학습률 스케줄러 스텝
    scheduler.step()

    # Early Stopping 체크
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

# Test 데이터 처리
test_combined_text = [' '.join([claim] + kw) for claim, kw in zip(test_claims, test_keywords)]
test_claim_embeddings = get_bert_embedding(test_claims)
test_keyword_embeddings = get_bert_embedding([' '.join(kw) for kw in test_keywords])

# Combine embeddings
test_combined_embeddings = np.concatenate((test_claim_embeddings, test_keyword_embeddings), axis=1)

# 추가 피처 생성 (예: claim 길이)
test_claim_lengths = np.array([len(claim.split()) for claim in test_claims]).reshape(-1, 1)
test_combined_embeddings = np.hstack([test_combined_embeddings, test_claim_lengths])

# 감정 점수 추가
test_sentiments = np.array([TextBlob(claim).sentiment.polarity for claim in test_claims]).reshape(-1, 1)
test_combined_embeddings = np.hstack([test_combined_embeddings, test_sentiments])

test_features = torch.tensor(test_combined_embeddings, dtype=torch.float)

# 테스트 그래프 생성 및 특성 추가
test_g = dgl.graph([], num_nodes=len(test))
test_g = dgl.add_self_loop(test_g)
test_g.ndata['feat'] = test_features

# 모델을 평가 모드로 전환
model.eval()

# 테스트 데이터에 대한 예측 수행
with torch.no_grad():
    test_logits = model(test_g, test_g.ndata['feat'])
    _, test_pred = torch.max(test_logits, dim=1)

# 예측 결과를 DataFrame으로 변환
submission = pd.DataFrame({
    'ID': test['id'],
    'Label': test_pred.numpy()
})

# 예측 결과를 CSV 파일로 저장
submission.to_csv('submission_improved__2.csv', index=False)

print("Predictions saved to submission_improved.csv")

Epoch 00000 | Loss 0.7664 | Train Accuracy 0.5335 | Val Accuracy 0.5322 | Val Loss 1.1304
Epoch 00001 | Loss 0.9819 | Train Accuracy 0.5134 | Val Accuracy 0.5028 | Val Loss 1.5478
Epoch 00002 | Loss 1.0607 | Train Accuracy 0.6103 | Val Accuracy 0.6203 | Val Loss 0.6617
Epoch 00003 | Loss 0.8310 | Train Accuracy 0.5821 | Val Accuracy 0.5881 | Val Loss 0.7631
Epoch 00004 | Loss 0.7014 | Train Accuracy 0.5894 | Val Accuracy 0.5767 | Val Loss 0.7079
Epoch 00005 | Loss 0.7667 | Train Accuracy 0.6558 | Val Accuracy 0.6430 | Val Loss 0.6324
Epoch 00006 | Loss 0.6080 | Train Accuracy 0.5456 | Val Accuracy 0.5294 | Val Loss 0.7366
Epoch 00007 | Loss 0.6510 | Train Accuracy 0.5444 | Val Accuracy 0.5379 | Val Loss 0.7589
Epoch 00008 | Loss 0.6641 | Train Accuracy 0.5745 | Val Accuracy 0.5530 | Val Loss 0.6808
Epoch 00009 | Loss 0.5647 | Train Accuracy 0.6465 | Val Accuracy 0.6212 | Val Loss 0.6478
Epoch 00010 | Loss 0.5773 | Train Accuracy 0.6709 | Val Accuracy 0.6373 | Val Loss 0.6336
Epoch 0001